In [1]:
from scipy import sparse as sps
import numpy as np

import nltk
nltk.download('reuters')
from nltk.corpus import reuters

[nltk_data] Downloading package reuters to
[nltk_data]     /Users/tomoki.otsuki/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [2]:
from sklearn.model_selection import train_test_split
from lda11.labelled_lda import LabelledLDA

In [3]:
file_ids = reuters.fileids()

In [4]:
X = []
y = []
for i, fid in enumerate(file_ids):
    doc = reuters.words(fid)
    X.append([word.lower() for word in doc])
    y.append(reuters.categories(fid))
    reuters.words(fid).close()

In [5]:
print(X[3])

['thai', 'trade', 'deficit', 'widens', 'in', 'first', 'quarter', 'thailand', "'", 's', 'trade', 'deficit', 'widened', 'to', '4', '.', '5', 'billion', 'baht', 'in', 'the', 'first', 'quarter', 'of', '1987', 'from', '2', '.', '1', 'billion', 'a', 'year', 'ago', ',', 'the', 'business', 'economics', 'department', 'said', '.', 'it', 'said', 'janunary', '/', 'march', 'imports', 'rose', 'to', '65', '.', '1', 'billion', 'baht', 'from', '58', '.', '7', 'billion', '.', 'thailand', "'", 's', 'improved', 'business', 'climate', 'this', 'year', 'resulted', 'in', 'a', '27', 'pct', 'increase', 'in', 'imports', 'of', 'raw', 'materials', 'and', 'semi', '-', 'finished', 'products', '.', 'the', 'country', "'", 's', 'oil', 'import', 'bill', ',', 'however', ',', 'fell', '23', 'pct', 'in', 'the', 'first', 'quarter', 'due', 'to', 'lower', 'oil', 'prices', '.', 'the', 'department', 'said', 'first', 'quarter', 'exports', 'expanded', 'to', '60', '.', '6', 'billion', 'baht', 'from', '56', '.', '6', 'billion', '.',

In [6]:
print(y[3])

['corn', 'grain', 'rice', 'rubber', 'sugar', 'tin', 'trade']


In [7]:
vocabulary = list(sorted({w for doc in X for w in doc}))
categories = list(sorted({cat for cats in y for cat in cats}))

vocabulary_to_index = { w:i for i, w in enumerate(vocabulary)}
category_to_index = { cat:i for i, cat in enumerate(categories)}

In [8]:
def codes_to_csr(codes, mapper):
    rows = []
    cols = []
    for i, row in enumerate(codes):
        for d in row:
            cols.append(mapper[d])
            rows.append(i)
    return sps.csr_matrix(
        (np.ones(len(rows), dtype=np.int32), (rows, cols)), shape=(len(codes), len(mapper))
    )

In [9]:
X = codes_to_csr(X, vocabulary_to_index)
y = codes_to_csr(y, category_to_index)

In [10]:
# include "common" tag.
# See e.g., https://shuyo.hatenablog.com/entry/20130228/lda
y = sps.hstack([y, np.ones((y.shape[0], 1))], format='csr')

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
# AD-LDA feature is available here, too
lda_model = LabelledLDA(n_iter=500, n_workers=4)
lda_model.fit(X_train, y_train)

100%|██████████| 500/500 [00:39<00:00, 12.54it/s]


In [13]:
phi = lda_model.phi.transpose()

In [14]:
for i, cat in enumerate(categories + ['COMMON']):
    print(f' ==== {cat} === ')
    rank = phi[i].argsort()[::-1]
    print([vocabulary[j] for j in rank[:30]])

 ==== acq === 
['.', 'the', ',', 'of', 'to', 'said', 'a', 'and', 'it', ';', '&', 'lt', 'in', '>', 'for', 'dlrs', 'its', 'company', 'mln', 'inc', 'shares', '-', 's', "'", 'corp', 'pct', 'has', 'stock', 'is', 'share']
 ==== alum === 
[',', 'aluminium', 'tonnes', '.', 'aluminum', '000', 'plant', 'alcan', 'smelter', 'mln', 'at', 'lme', 'company', 'prices', 'of', 'contract', '1', 'capacity', 'metal', 'alumina', 'february', 'alcoa', 'cbt', 'primary', 'spokesman', 'ltd', '&', 'lt', 'says', ';']
 ==== barley === 
[',', '.', 'barley', 'of', 'for', '-', '000', 'tonnes', '--', 'ecus', 'export', 'the', 'wheat', 'at', 'ec', 'and', 'a', 'free', 'maize', 'market', 'said', 'licences', 'tonne', 'v', 'trade', 'french', 'non', 'rebate', '65', 'saudi']
 ==== bop === 
['.', 'billion', 'in', 'the', ',', 'a', 'of', 'dlrs', 'mln', 'deficit', 'surplus', 'to', 'from', 'account', 'february', 'current', '2', 'trade', 'january', 'and', '1986', '1', 'year', 's', 'said', 'quarter', '4', '7', '6', 'with']
 ==== carca

In [15]:
theta_test = lda_model.transform(X_test, gibbs_burn_in=5, n_iter=100, n_workers=4)

# remove "common" and renormalize
theta_test = theta_test[:, :-1] / theta_test[:, :-1].sum(axis=1)[:, np.newaxis]

In [16]:
# constant model
constant_predictor = y_train[:, :-1].sum(axis=0).astype(np.float64).A1 / y_train[:, :-1].sum()
constant_predictor += 1e-15
constant_predictor /= constant_predictor.sum()

In [17]:
# vs top popular
print(
    'log_loss of contant model: {:.3f}'.format(
        -(np.log(constant_predictor)[np.newaxis, :] * y_test[:, :-1].toarray()).sum(axis=1).mean()        
    )
)

print(
    'log_loss of labelled lda theta: {:.3f}'.format(
        - (np.log(theta_test) * y_test[:, :-1].toarray()).sum(axis=1)[-3]
    )
)

log_loss of contant model: 3.570
log_loss of labelled lda theta: 1.748
